# Web Scraping

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

In [2]:
# Função para extrair os títulos dos itens
def get_itens_title(doc):
    title_tags = doc.find_all('a', class_='poly-component__title')
    return [tags.text.strip() for tags in title_tags]

# Função para extrair os preços dos itens
def get_price(doc):
    price_tags = doc.find_all('div', class_='poly-price__current')
    prices = []
    for tag in price_tags:
        # Busca as partes do preço: fração e centavos
        fraction = tag.find('span', class_='andes-money-amount__fraction')
        cents = tag.find('span', class_='andes-money-amount__cents')
        # Monta o preço completo caso ambas as partes existam
        if fraction and cents:
            prices.append(f"{fraction.text}.{cents.text}")
        # Caso só exista a fração, retorna somente ela
        elif fraction:
            prices.append(fraction.text)
        # Se não houver fração, considera o preço como 0.00
        else:
            prices.append("0.00")
    return prices

# Função para carregar e processar o HTML de uma página específica
def get_doc(url):
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception(f'Failed to load page {url}')
    return BeautifulSoup(response.text, 'html.parser')

# Função principal para raspar múltiplas páginas
def scrape_multiple_pages(n):
    base_url = 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page='
    titles, prices = [], []
    
    for page in range(1, n + 1):
        print(f"Scraping page {page}...") 
        doc = get_doc(base_url + str(page))

        titles.extend(get_itens_title(doc))
        prices.extend(get_price(doc))
    
        min_length = min(len(titles), len(prices))
        titles = titles[:min_length]
        prices = prices[:min_length]
    
    return pd.DataFrame({'TITLE': titles, 'PRICE': prices})

df_mercado_livre = scrape_multiple_pages(10)

df_mercado_livre["SCRAPY_DATETIME"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

df_mercado_livre['PRICE'] = pd.to_numeric(df_mercado_livre['PRICE'].str.replace(',', '.'))

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...


In [3]:
df_mercado_livre

,TITLE,PRICE,SCRAPY_DATETIME
0,Creatina Monohidratada 500g Soldiers Nutrition...,64.690,2024-12-17 17:51:13
1,Kit 6 Pares Meias Lupo Original Cano Médio Alt...,59.300,2024-12-17 17:51:13
2,Tênis Nike Court Legacy Masculino,289.990,2024-12-17 17:51:13
3,Cortina Oxford Tecido Grosso 300x250 Sala Quarto,57.380,2024-12-17 17:51:13
4,Caixa De Som Partybox Aiwa Pb-06 Bluetooth 20h...,1.499,2024-12-17 17:51:13
...,...,...,...
534,Cooktop 5 Bocas Preto Com Mesa Vidro Atlas Agi...,489.900,2024-12-17 17:51:13
535,Relógio Feminino+ Colar Pingente Brincos Folhe...,56.920,2024-12-17 17:51:13
536,Pijama Macacão Menina Infantil Kigurumi De Pel...,147.770,2024-12-17 17:51:13
537,Smartphone Motorola Moto G54 5g 256 Gb Azul 8 ...,1.169,2024-12-17 17:51:13


# Armazenando os dados no banco de dados

In [4]:
from sqlalchemy import create_engine
import sqlite3

In [5]:
mercado_livre_schema = """
CREATE TABLE mercado_livre (
    TITLE              TEXT,
    PRICE              INTEGER,
    SCRAPY_DATETIME    TEXT
)
"""

In [6]:
conn = create_engine('sqlite:///mercado_livre.sqlite', echo=False)
conn = sqlite3.connect('mercado_livre.sqlite')

In [7]:
df_mercado_livre.to_sql('mercado_livre', con=conn, if_exists='append', index=False)

539

In [8]:
query = """
SELECT * FROM mercado_livre
"""
df_mercado_livre = pd.read_sql_query(query, conn)

In [9]:
df_mercado_livre[df_mercado_livre["TITLE"] == "Caixa De Som Partybox Aiwa Pb-06 Bluetooth 20h Rgb Usb Tws"]

,TITLE,PRICE,SCRAPY_DATETIME
4,Caixa De Som Partybox Aiwa Pb-06 Bluetooth 20h...,1.499,2024-12-17 17:43:43
542,Caixa De Som Partybox Aiwa Pb-06 Bluetooth 20h...,1.499,2024-12-17 17:43:43
1080,Caixa De Som Partybox Aiwa Pb-06 Bluetooth 20h...,1.499,2024-12-17 17:51:13
